### 🧠 Phân tích và Đánh giá Mô hình K-Nearest Neighbors (KNN)

#### 1. Tổng quan lý thuyết KNN
K-Nearest Neighbors (KNN) là một thuật toán **học có giám sát (supervised learning)**, thường được sử dụng cho các bài toán **phân loại (classification)** và **hồi quy (regression)**.  
Nguyên lý hoạt động rất trực quan: một điểm dữ liệu mới sẽ được gán nhãn dựa trên **các điểm “hàng xóm” gần nhất** trong tập huấn luyện.

Khoảng cách giữa các điểm thường được đo bằng các độ đo như:
- **Euclidean Distance (khoảng cách Euclid)**
- **Manhattan Distance (tổng độ chênh tuyệt đối)**
- **Minkowski Distance**  

Trong đoạn mã này, khoảng cách **Manhattan** được sử dụng:
\[
d(x_1, x_2) = \sum_{i=1}^{n} |x_{1i} - x_{2i}|
\]

Thuật toán sẽ chọn ra **K điểm gần nhất**, sau đó dùng **bỏ phiếu đa số (majority voting)** để xác định nhãn đầu ra.

---

#### 2. Cách triển khai trong đoạn mã
Đoạn code triển khai KNN **thủ công (from scratch)** để minh họa rõ quy trình hoạt động của thuật toán, bao gồm các bước chính sau:

- **Chuẩn bị dữ liệu:**
  Dữ liệu được chia thành **5 folder (folder₁ → folder₅)**, dùng cho quá trình **5-fold cross-validation**.  
  Mỗi vòng, một folder được chọn làm **test set**, 4 folder còn lại làm **train set**.

- **Chuẩn hóa dữ liệu (Normalization):**
  Áp dụng phép **Min–Max scaling** đưa các đặc trưng về khoảng [0, 1] để đảm bảo công bằng khi tính khoảng cách.

- **Hàm `manhattan_distance`:**
  Tính tổng giá trị tuyệt đối giữa từng cặp đặc trưng của hai mẫu dữ liệu.

- **Hàm `knn_predict`:**
  1. Tính khoảng cách Manhattan giữa mẫu test và toàn bộ mẫu train.  
  2. Chọn **k điểm gần nhất**.  
  3. Thực hiện **bỏ phiếu đa số** để xác định nhãn dự đoán.

- **Đánh giá mô hình:**
  Với mỗi fold, chương trình tính **độ chính xác (Accuracy)** bằng công thức:
  \[
  \text{Accuracy} = \frac{\text{Số dự đoán đúng}}{\text{Tổng số mẫu}}
  \]
  Kết quả từng fold được lưu trong danh sách `results` và xuất ra file CSV.

---

#### 3. Nhận xét và Đánh giá
**Ưu điểm:**
- Cấu trúc mã rõ ràng, dễ hiểu, minh họa chính xác cơ chế của KNN.  
- Việc **không sử dụng thư viện sklearn** giúp thể hiện rõ bản chất thuật toán.  
- Sử dụng **cross-validation 5-fold** giúp đánh giá tổng quát độ ổn định mô hình.  
- Chuẩn hóa dữ liệu đúng cách giúp tránh sai lệch do thang đo đặc trưng.

**Hạn chế:**
- Chi phí tính toán cao do phải đo khoảng cách đến toàn bộ mẫu train cho mỗi dự đoán.  
- Chưa có cơ chế tối ưu như **KD-Tree** hay **Ball-Tree**.  
- Giá trị `k` cố định (k=3) chưa được tinh chỉnh bằng grid search hoặc cross-validation sâu hơn.

---

#### 4. Kết luận
Mô hình **KNN (k=3, Manhattan distance)** cho kết quả ổn định và phù hợp với bài toán **phân loại Iris**.  
Phương pháp **5-fold cross-validation** cung cấp đánh giá khách quan về độ chính xác trung bình.  
Đoạn mã là một ví dụ tiêu biểu cho việc **hiểu bản chất thuật toán học máy cơ bản**, tạo nền tảng để tiến tới các mô hình phức tạp hơn như SVM, Decision Tree hay Neural Network.


In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import MinMaxScaler


# === Hàm tính khoảng cách Manhattan ===
def manhattan_distance(x1, x2):
    return np.sum(np.abs(x1 - x2))

# === Hàm dự đoán bằng KNN ===
def knn_predict(test_sample, train_features, train_labels, k=3):
    # Tính khoảng cách từ test_sample đến tất cả train
    distances = [manhattan_distance(test_sample, train_features.iloc[i]) 
                 for i in range(len(train_features))]
    
    # Lấy chỉ số của k điểm gần nhất
    k_indices = np.argsort(distances)[:k]
    k_labels = train_labels.iloc[k_indices]
    
    # Bỏ phiếu đa số
    most_common = Counter(k_labels).most_common(1)[0][0]
    return most_common

# === Thư mục dữ liệu và kết quả ===
base_dir = "/media/pphong/D:/ML & DL/ML/AI_02/iris_folder"
result_dir = "/media/pphong/D:/ML & DL/ML/AI_02/result_folder"
os.makedirs(result_dir, exist_ok=True)

# === Danh sách folder ===
folders = [f"folder_{i}" for i in range(1, 6)]
results = []   # Lưu accuracy từng lần
k = 3          # Số láng giềng gần nhất

# === Cross-validation 5-fold ===
for test_folder in folders:
    print(f"\n===== Test với {test_folder} (k={k}) =====")

    # Test data
    test_path = os.path.join(base_dir, test_folder)
    test_csv = [f for f in os.listdir(test_path) if f.endswith('.csv')]
    if not test_csv:
        raise FileNotFoundError(f"Không tìm thấy file CSV trong {test_path}")
    test_df = pd.read_csv(os.path.join(test_path, test_csv[0]))

    # Train data (4 folder còn lại)
    train_folders = [f for f in folders if f != test_folder]
    train_dfs = []
    for f in train_folders:
        csvs = [c for c in os.listdir(os.path.join(base_dir, f)) if c.endswith('.csv')]
        if not csvs:
            raise FileNotFoundError(f"Không tìm thấy CSV trong {f}")
        train_dfs.append(pd.read_csv(os.path.join(base_dir, f, csvs[0])))
    train_df = pd.concat(train_dfs, ignore_index=True)

    # === Chuẩn hóa dữ liệu (0–1) theo train ===
    X_train = train_df.drop(columns=['Id', 'Species'])
    y_train = train_df['Species']
    X_test = test_df.drop(columns=['Id', 'Species'])
    y_test = test_df['Species']

    X_train_norm = (X_train - X_train.min()) / (X_train.max() - X_train.min())
    X_test_norm = (X_test - X_train.min()) / (X_train.max() - X_train.min())

    # === Dự đoán ===
    predictions = [knn_predict(X_test_norm.iloc[i], X_train_norm, y_train, k=k) 
                   for i in range(len(X_test_norm))]

    # === Tính accuracy ===
    correct = sum(1 for i in range(len(y_test)) if predictions[i] == y_test.iloc[i])
    total = len(y_test)
    accuracy = correct / total

    results.append(accuracy)
    print(f"Accuracy = {accuracy:.3f} ({correct}/{total} đúng)")

    # Lưu kết quả
    result_df = test_df.copy()
    result_df['Predicted'] = predictions
    result_path = os.path.join(result_dir, f"result_{test_folder}_k{k}.csv")
    result_df.to_csv(result_path, index=False)
    print(f"Đã lưu: {result_path}")

# === Tổng kết ===
print("\n=== KẾT QUẢ 5-FOLD CROSS-VALIDATION ===")
for i, acc in enumerate(results, 1):
    print(f"Fold {i}: Accuracy = {acc:.3f}")
print(f"Mean accuracy = {np.mean(results):.3f}")



===== Test với folder_1 (k=3) =====
Accuracy = 0.967 (29/30 đúng)
Đã lưu: /media/pphong/D:/ML & DL/ML/AI_02/result_folder/result_folder_1_k3.csv

===== Test với folder_2 (k=3) =====
Accuracy = 1.000 (30/30 đúng)
Đã lưu: /media/pphong/D:/ML & DL/ML/AI_02/result_folder/result_folder_2_k3.csv

===== Test với folder_3 (k=3) =====
Accuracy = 0.933 (28/30 đúng)
Đã lưu: /media/pphong/D:/ML & DL/ML/AI_02/result_folder/result_folder_3_k3.csv

===== Test với folder_4 (k=3) =====
Accuracy = 0.967 (29/30 đúng)
Đã lưu: /media/pphong/D:/ML & DL/ML/AI_02/result_folder/result_folder_4_k3.csv

===== Test với folder_5 (k=3) =====
Accuracy = 0.967 (29/30 đúng)
Đã lưu: /media/pphong/D:/ML & DL/ML/AI_02/result_folder/result_folder_5_k3.csv

=== KẾT QUẢ 5-FOLD CROSS-VALIDATION ===
Fold 1: Accuracy = 0.967
Fold 2: Accuracy = 1.000
Fold 3: Accuracy = 0.933
Fold 4: Accuracy = 0.967
Fold 5: Accuracy = 0.967
Mean accuracy = 0.967
